Вы получили основные навыки обработки данных, теперь пора испытать их на практике. Сейчас вам предстоит заняться задачей классификации.

Представлен датасет центра приюта животных, и вашей задачей будет обучить модель таким образом, чтобы  по определенным признакам была возможность максимально уверенно предсказать метки 'Adoption' и 'Transfer' (столбец “outcome_type”).

Здесь вы вольны делать что угодно. Я хочу видеть от вас:
1. Проверка наличия/обработка пропусков
2. Проверьте взаимосвязи между признаками
3. Попробуйте создать свои признаки
4. Удалите лишние
5. Обратите внимание на текстовые столбцы. Подумайте, что можно извлечь полезного оттуда
6. Использование профайлера вам поможет.
7. Не забывайте, что у вас есть PCA (Метод главных компонент). Он может пригодиться.

Вспомните о всем, что я говорил на предыдущих занятиях. Не все будет пригодится, но в жизни вам никто не будет говорить, что использовать :)

Хорошим классификатором для этой задачи будет "Случайный лес" (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

Понимать суть работы "леса" не обязательно на данном этапе, но качество предсказаний будет выше, чем с линейным классификатором. (если желаете, вот гайд https://adataanalyst.com/scikit-learn/linear-classification-method/)

Желаю успеха :)

> спасибо =)

## Начало работы

Начнём с того, что подготовим себе окружение и загрузим данные

In [334]:
%matplotlib inline

import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import re
import datetime as dt
from functools import reduce

Данные предоставлены в файле "aac_shelter_outcomes.csv"

In [335]:
df = pd.read_csv('./data/aac_shelter_outcomes.csv')
df.shape

(78256, 12)

Отлично, данных у нас предостаточно. 78к наблюдений по 12 признакам достаточно.

In [336]:
df.head()

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male
4,5 months,A683115,Other,Bat Mix,Brown,2014-01-07T00:00:00,2014-07-07T14:04:00,2014-07-07T14:04:00,NaN,Rabies Risk,Euthanasia,Unknown


По условиям задачи, нам надо предсказывать признаки 'Adoption' и 'Transfer' (столбец “outcome_type”). Но данный столбец содержит больше признаков. Проверим все возможные значения в столбце

In [337]:
df.outcome_type.unique()

array(['Transfer', 'Adoption', 'Euthanasia', 'Return to Owner', 'Died',
       'Disposal', 'Relocate', 'Missing', nan, 'Rto-Adopt'], dtype=object)

Получается, что нужных нам значений меньшинство... Проверим, сколько в процентном соотношении занимают нужные нам значения от всего датасета

In [338]:
len(df[df['outcome_type'].isin(['Transfer', 'Adoption'])]) / len(df) * 100

72.34077898180331

Нужные нам значения находятся у 72% записей, а это значит, что можно остальные данные можно отбросить и дифицита данных у нас не будет

In [339]:
df = df[df['outcome_type'].isin(['Transfer', 'Adoption'])]
df.shape

(56611, 12)

Теперь попробуем дать описание каждой колонки:
- age_upon_outcome - возраст животного при выходе из приюта 
- animal_id - идентификатор животного
- animal_type - вид животного
- breed - порода
- color - цвет
- date_of_birth - дата рождения животного (оценивается примерно)
- datetime - дата и время выхода животного из приюта
- monthyear - месяц и год выхода животного из приюта
- name - кличка животного
- outcome_subtype - более конкретное описание причины покидания приюта
- outcome_type - как именно животное покинуло приют
- sex_upon_outcome - пол животного и признак сохранения репродуктивной функции

Теперь у нас есть хоть какая-то ясность о представленных данных. Далее - я буду анализировать, какой из признаков нам будет полезен или что мы можем извлечь из каких-либо признаков

### age_upon_outcome
Из данной переменной можно извлечь очень полезный показатель - возраст животного в момент его прощания с приютом. Как правило, из приюта люди стремятся забрать более молодое животное, что сильно повлиять на результаты обучения модели.

Данный признак я извлеку в виде значения в днях

### animal_id
Локальный идентификатор животного, никакой смысловой нагрузки не несёт

Данный признак я удалю

### animal_type
С большей охотой люди в городе готовы забирать кошек и собак. Я считаю, что у них больше шансов найти новый дом и хозяев

Данную переменную следует преобразовать через OneHotEncoding

### breed
Из данного признака можно извлечь очень важный показатель - является ли митисом животное.

Как правило, люди подвержены забирать более породистых животных. Остальные породы будут очищены от признака "mix" и будут обработаны при помощи OneHotEncoding

### color
Как ни странно, цвет шерсти\оперения\чешуи животного может сыграть очень важную роль в его судьбе. Это навеяно нам фильмами, литературой, рекламой.

Цвет животного я преобразую при помощи OneHotEncoding

### date_of_birth
Несомненно важный признак, но я уже решил вытаскивать данные о возрасте животного в момент его прощания с приютом из другого признака

Данные признак я удалю

### datetime
Данный признак может указать на сезонный "спрос" на бездомных животных. Так же у нас есть признак "monthyear" который поможет установить более точную дату забора животного из приюта.

### monthyear
Полная копия признака "datetime"

Данный признак я удалю

### name
Как правило, люди любят давать клички самостоятельно и этот признак вряд ли сыграет определяющую роль.

Данный признак я удалю

### outcome_subtype
Мы отказались от некоторых из "развязок" судьбы животных. Если в решении об "эвтаназии" были такие диагнозы, как подзрение на бешенство, что определяло 100% эвтаназию, то после чистки я не могу с уверенность сказать, что может показать данный признак.

Признак нуждается в исследовании

### outcome_type
Целевое значение. Нашей задачей является определить 'Adoption'(нашлись хозяева) или 'Transfer'(животное было перемещено). Логичнее всего из данного столбца сделать булевое значение 'is_adopted' 0 или 1 

### sex_upon_outcome
Отличная переменная! Она даёт нам сразу два значения - пол животного и признак сохранения его репродуктивной функции. Т.к. датасет у нас городской, то в городе есть статистика, утверждающая, что из приюта животных берут люди НЕ для разведения. И животное, желательно, брать уже кастрированным/стерилизованным.

Человеку "ЗА" выбор кастрированного/стерилизованного животного говорит сразу несколько фактов:
1. они более спокойны
1. на поведение не будет влиять сезон
1. не придётся ездить на "случки"
1. меньше запахов по дому
1. не придётся тратиться на операцию (в рф средняя стоимость кастрации - 1000-1500 рублей, а стерилизации 2000-12000 рублей в зависимости от размеров животного. За рубежом же эта сумма может быть в 10 раз выше)
1. животные в приюте не имеют паспорта и родословной, так что разводить их бессмысленно в коммерческом смысле

Так что, из этой переменной я извлеку признак о сохранности репродуктивных органов и а к полу применю OneHotEncoding


## Подготовка данных

Начнём с "age_upon_outcome" и сперва проверим из каких ункальных значений состоит эта переменная

In [340]:
age_upon_outcome_unique = df.age_upon_outcome.unique()
age_upon_outcome_unique

array(['2 weeks', '1 year', '9 years', '4 months', '3 years', '1 month',
       '3 months', '2 years', '2 months', '3 weeks', '8 months',
       '5 months', '12 years', '4 years', '7 years', '5 years', '5 days',
       '10 months', '4 weeks', '2 days', '10 years', '6 months',
       '8 years', '11 months', '15 years', '7 months', '6 years',
       '16 years', '9 months', '6 days', '4 days', '1 week', '3 days',
       '14 years', '13 years', '1 day', '1 weeks', '0 years', '11 years',
       '5 weeks', '20 years', '17 years', '19 years', '18 years',
       '25 years', nan], dtype=object)

Каждое из этих значений я могу преобразовать к какому-то множителю, на который помножу числовое значение рядом, находящееся в признаке.

In [341]:
def calc_age_in_days(age_upon_outcome):    
    if age_upon_outcome is np.nan:
        return 0
    
    count, period = age_upon_outcome.split()
    multiplier = 0
    
    if 'day' in period:
        multiplier = 1
    elif 'week' in period:
        multiplier = 7
    elif 'month' in period:
        multiplier = 30
    elif 'year' in period:
        multiplier = 365
        
    return int(count) * multiplier

df['days_upon_outcome'] = df.age_upon_outcome.apply(calc_age_in_days)

df.head()

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome,days_upon_outcome
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male,14
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female,365
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male,365
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male,3285
5,4 months,A664462,Dog,Leonberger Mix,Brown/White,2013-06-03T00:00:00,2013-10-07T13:06:00,2013-10-07T13:06:00,*Edgar,Partner,Transfer,Intact Male,120


В описании датасета говорилось, что данный признак указывался "примерно", но мы то знаем, что это была оценка "на глазок"... Поэтому такие не строгие показатели будут нормой для обучения модели.

Теперь извлечём признак метиса из поля breed. Это можно сделать обращаясь по признаку "mix".

In [342]:
len(df.breed.unique())

1803

Различных возможных значений оказалось куда больше, чем я ожидал. Проанализирую слова, составляющие данную колонку

In [343]:
breed_words = []

for words_list in df.breed.str.lower().str.split():
    for word in words_list:
        breed_words.append(word)
        
len(breed_words)

165465

In [344]:
df_breed_words = pd.DataFrame(breed_words)
len(df_breed_words[0].unique())

1348

In [345]:
df_breed_words[0].value_counts().head(20)

mix           48939
shorthair     25391
domestic      24391
chihuahua      4364
labrador       4280
retriever      4240
bull           4117
pit            3921
terrier        2571
shepherd       2315
hair           2224
medium         2180
german         1693
australian     1682
longhair       1452
cattle         1238
dog            1153
miniature      1045
siamese         909
dachshund       867
Name: 0, dtype: int64

Оценив содержание слов в породе животного, можно выделить ещё такие значения, как длину шерсти. Это добавит дополнительные три категории: "shorthair", "medium hair" и "longhair"

Сперва выделим признак "is_mix"

In [346]:
MIX_ATTR = 'mix'
SHORT_HAIR_ATTR = 'shorthair'
MEDIUM_HAIR_ATTR = 'medium hair'
LONG_HAIR_ATTR = 'longhair'

df['breed'] = df.breed.str.lower()

In [347]:
df['is_mix'] = df.breed.str.contains(MIX_ATTR).astype(int)
df.head()

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome,days_upon_outcome,is_mix
0,2 weeks,A684346,Cat,domestic shorthair mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male,14,1
1,1 year,A666430,Dog,beagle mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female,365,1
2,1 year,A675708,Dog,pit bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male,365,0
3,9 years,A680386,Dog,miniature schnauzer mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male,3285,1
5,4 months,A664462,Dog,leonberger mix,Brown/White,2013-06-03T00:00:00,2013-10-07T13:06:00,2013-10-07T13:06:00,*Edgar,Partner,Transfer,Intact Male,120,1


In [348]:
df['l_hair'] = df.breed.str.contains(LONG_HAIR_ATTR).astype(int)
df['m_hair'] = df.breed.str.contains(MEDIUM_HAIR_ATTR).astype(int)
df['s_hair'] = df.breed.str.contains(SHORT_HAIR_ATTR).astype(int)

df.head()

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome,days_upon_outcome,is_mix,l_hair,m_hair,s_hair
0,2 weeks,A684346,Cat,domestic shorthair mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male,14,1,0,0,1
1,1 year,A666430,Dog,beagle mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female,365,1,0,0,0
2,1 year,A675708,Dog,pit bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male,365,0,0,0,0
3,9 years,A680386,Dog,miniature schnauzer mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male,3285,1,0,0,0
5,4 months,A664462,Dog,leonberger mix,Brown/White,2013-06-03T00:00:00,2013-10-07T13:06:00,2013-10-07T13:06:00,*Edgar,Partner,Transfer,Intact Male,120,1,0,0,0


[мысли вслух]

Теперь у нас есть одна проблема, которую я не предсказал изначально, приступая к работе над полем "breed". У нас есть 3 категории, где удалось определить шерсть и множество записей, где не удалось определить тип шерсти. Мне кажется, что такие записи автоматом попадают в 4 категорию. Не буду добавлять четвёртый случай, нет определенной шерсти.

Теперь не плохо было бы отчистить наше поле "breed" от тех слов, которые мы взяли на предыдущих нескольких шагах, чтобы сократить кол-во категорий, возможно взятых из этого поля именно по породам.

In [349]:
df.breed = df.breed.str.replace(f'{MIX_ATTR}|{LONG_HAIR_ATTR}|{SHORT_HAIR_ATTR}|{MEDIUM_HAIR_ATTR}', '').str.strip()
df.head()

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome,days_upon_outcome,is_mix,l_hair,m_hair,s_hair
0,2 weeks,A684346,Cat,domestic,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male,14,1,0,0,1
1,1 year,A666430,Dog,beagle,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female,365,1,0,0,0
2,1 year,A675708,Dog,pit bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male,365,0,0,0,0
3,9 years,A680386,Dog,miniature schnauzer,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male,3285,1,0,0,0
5,4 months,A664462,Dog,leonberger,Brown/White,2013-06-03T00:00:00,2013-10-07T13:06:00,2013-10-07T13:06:00,*Edgar,Partner,Transfer,Intact Male,120,1,0,0,0


In [350]:
len(df.breed.unique())

1578

После нехитрых преобразований мы получили из 1803 уникальных значений породы 1578. Не шикарный, но успех.

К сожалению, такое количество категорий это, мягко говоря, перебор. Поэтому я продолжу изучать данную категорию

In [351]:
breed_v_counts = df.breed.value_counts()

breed_v_counts.head(50)

domestic                              24371
chihuahua                              3790
pit bull                               3614
labrador retriever                     3391
german shepherd                        1377
siamese                                 909
australian cattle dog                   841
dachshund                               728
boxer                                   485
border collie                           483
miniature poodle                        425
catahoula                               356
jack russell terrier                    327
rat terrier                             326
australian shepherd                     320
yorkshire terrier                       303
beagle                                  295
cairn terrier                           284
siberian husky                          270
pointer                                 270
miniature schnauzer                     270
rabbit sh                               269
great pyrenees                  

In [352]:
breed_v_counts.tail(50)

australian cattle dog/chow chow                   1
labrador retriever/english pointer                1
miniature pinscher/maltese                        1
rat terrier/australian cattle dog                 1
australian cattle dog/black mouth cur             1
rat terrier/queensland heeler                     1
whippet/anatol shepherd                           1
rat terrier/pug                                   1
papillon/yorkshire terrier                        1
mouse                                             1
akita/mastiff                                     1
german wirehaired pointer/labrador retriever      1
whippet/catahoula                                 1
standard schnauzer/soft coated wheaten terrier    1
rhod ridgeback/australian cattle dog              1
dalmatian/pointer                                 1
smooth fox terrier/labrador retriever             1
st. bernard rough coat/border collie              1
plott hound/border collie                         1
rottweiler/b

К сожалению, я не думаю что создание и балансировка такого огромного числа категорий принесёт нам пользу, поэтому в дальнейшем поле "breed" я удалю

Теперь можно приступить к изучению признака outcome_subtype

In [353]:
outcome_subtype = df.outcome_subtype.unique()

outcome_subtype.shape, outcome_subtype

((7,), array(['Partner', nan, 'Offsite', 'Foster', 'SCRP', 'Barn', 'Snr'],
       dtype=object))

Возможных значений в поле не много, поэтому его можно преобразовать при помощи OneHotEncoding

Окей, теперь пришла пора преобразовать наше целевое значение

In [354]:
df['is_adopted'] = df.outcome_type.str.lower().str.contains('adopt').astype(int)
df.head()

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome,days_upon_outcome,is_mix,l_hair,m_hair,s_hair,is_adopted
0,2 weeks,A684346,Cat,domestic,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male,14,1,0,0,1,0
1,1 year,A666430,Dog,beagle,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female,365,1,0,0,0,0
2,1 year,A675708,Dog,pit bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male,365,0,0,0,0,1
3,9 years,A680386,Dog,miniature schnauzer,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male,3285,1,0,0,0,0
5,4 months,A664462,Dog,leonberger,Brown/White,2013-06-03T00:00:00,2013-10-07T13:06:00,2013-10-07T13:06:00,*Edgar,Partner,Transfer,Intact Male,120,1,0,0,0,0


Ну и последнее преобразование - получение признака сохранности репродуктивных органов и пола

In [355]:
sex_upon_outcome = df.sex_upon_outcome.unique()

sex_upon_outcome.shape, sex_upon_outcome

((5,), array(['Intact Male', 'Spayed Female', 'Neutered Male', 'Intact Female',
        'Unknown'], dtype=object))

"intact" - означает сохранение репродуктивной функции, а "spayed" и "neutered" - лишение

In [356]:
df['sex_upon_outcome'] = df.sex_upon_outcome.str.lower()

In [357]:
df['sex_intact'] = df.sex_upon_outcome.str.contains('intact').astype(int)

df.head()

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome,days_upon_outcome,is_mix,l_hair,m_hair,s_hair,is_adopted,sex_intact
0,2 weeks,A684346,Cat,domestic,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,intact male,14,1,0,0,1,0,1
1,1 year,A666430,Dog,beagle,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,spayed female,365,1,0,0,0,0,0
2,1 year,A675708,Dog,pit bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,neutered male,365,0,0,0,0,1,0
3,9 years,A680386,Dog,miniature schnauzer,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,neutered male,3285,1,0,0,0,0,0
5,4 months,A664462,Dog,leonberger,Brown/White,2013-06-03T00:00:00,2013-10-07T13:06:00,2013-10-07T13:06:00,*Edgar,Partner,Transfer,intact male,120,1,0,0,0,0,1


In [358]:
df['sex'] = df.sex_upon_outcome.str.replace('intact|spayed|neutered', '').str.strip()

df.head()

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome,days_upon_outcome,is_mix,l_hair,m_hair,s_hair,is_adopted,sex_intact,sex
0,2 weeks,A684346,Cat,domestic,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,intact male,14,1,0,0,1,0,1,male
1,1 year,A666430,Dog,beagle,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,spayed female,365,1,0,0,0,0,0,female
2,1 year,A675708,Dog,pit bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,neutered male,365,0,0,0,0,1,0,male
3,9 years,A680386,Dog,miniature schnauzer,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,neutered male,3285,1,0,0,0,0,0,male
5,4 months,A664462,Dog,leonberger,Brown/White,2013-06-03T00:00:00,2013-10-07T13:06:00,2013-10-07T13:06:00,*Edgar,Partner,Transfer,intact male,120,1,0,0,0,0,1,male


Чтож... Пришло время сбросить балласт и дропнуть не нужные нам столбцы

In [359]:
df.drop(['age_upon_outcome', 'animal_id', 'date_of_birth', 'monthyear', 'name', 'outcome_type', 'sex_upon_outcome', 'breed'], axis=1, inplace=True)

In [360]:
df.head()

,animal_type,color,datetime,outcome_subtype,days_upon_outcome,is_mix,l_hair,m_hair,s_hair,is_adopted,sex_intact,sex
0,Cat,Orange Tabby,2014-07-22T16:04:00,Partner,14,1,0,0,1,0,1,male
1,Dog,White/Brown,2013-11-07T11:47:00,Partner,365,1,0,0,0,0,0,female
2,Dog,Blue/White,2014-06-03T14:20:00,NaN,365,0,0,0,0,1,0,male
3,Dog,White,2014-06-15T15:50:00,Partner,3285,1,0,0,0,0,0,male
5,Dog,Brown/White,2013-10-07T13:06:00,Partner,120,1,0,0,0,0,1,male


Преобразую все строковые значения к нижнему регистру, для бОльшей чистоты эксперемента

In [361]:
df['animal_type'] = df['animal_type'].str.lower()
df['color'] = df['color'].str.lower()
df['outcome_subtype'] = df['outcome_subtype'].str.lower()

Я забыл об одном нестандартном для меня признаке - о времени, когда сменился статус по животному. Изначально оно предоставлено в виде даты, но регрессию по датам делать нельзя. Поэтому я приведу даты к более простому, числовому значению

In [362]:
df['datetime'] = pd.to_datetime(df.datetime).map(dt.datetime.toordinal)

In [363]:
df.head()

,animal_type,color,datetime,outcome_subtype,days_upon_outcome,is_mix,l_hair,m_hair,s_hair,is_adopted,sex_intact,sex
0,cat,orange tabby,735436,partner,14,1,0,0,1,0,1,male
1,dog,white/brown,735179,partner,365,1,0,0,0,0,0,female
2,dog,blue/white,735387,NaN,365,0,0,0,0,1,0,male
3,dog,white,735399,partner,3285,1,0,0,0,0,0,male
5,dog,brown/white,735148,partner,120,1,0,0,0,0,1,male


Отлично! Теперь нужно подготовить всё к OneHotEncoding преобразованию

Для этого воспользуемся профайлером и уберём "битые" значения

In [364]:
df.profile_report(style={ 'full_width': True })

Профайлер - наш хороший друг! При помощи него я понял что поле "outcome_subtype" может испортить нам все результаты, ведь в нём слишком много пропущенных значений и это может "утянуть" результаты в непредсказуемую сторону. Я его удалю

Вдобавок, я выяснил, что поле "color" имеет слишком много различных вариантов... Его надо дорабатывать

In [366]:
df.drop(['outcome_subtype'], axis=1, inplace=True)

In [367]:
df.head()

,animal_type,color,datetime,days_upon_outcome,is_mix,l_hair,m_hair,s_hair,is_adopted,sex_intact,sex
0,cat,orange tabby,735436,14,1,0,0,1,0,1,male
1,dog,white/brown,735179,365,1,0,0,0,0,0,female
2,dog,blue/white,735387,365,0,0,0,0,1,0,male
3,dog,white,735399,3285,1,0,0,0,0,0,male
5,dog,brown/white,735148,120,1,0,0,0,0,1,male


In [368]:
df.color.value_counts()

black/white           6111
black                 5141
brown tabby           3975
brown tabby/white     2083
orange tabby          1914
                      ... 
red tick/blue tick       1
liver/cream              1
cream/blue point         1
liver/chocolate          1
tortie/brown             1
Name: color, Length: 475, dtype: int64

In [375]:
splited_colors_series = df.color.str.split('/')
max(map(len, splited_colors_series))

2

Сочетание цветов максимальной длины - 2. Думаю, возможно будет сделать так, чтобы одну запись можно было отнести к двум категориальным признакам

In [380]:
splited_colors = []

for colors_list in splited_colors_series:
    for color in colors_list:
        splited_colors.append(color)

In [385]:
len(pd.DataFrame(splited_colors)[0].unique())

57

Итого количество категорий может сократиться до 57. Это хороший результат и это нас устроит. Теперь нужно придумать алгоритм, который может проставить две категории одной записи

In [401]:
df['color_common'] = df.color.map(lambda colors: colors[0])
df['color_second'] = df.color.map(lambda colors: colors[1] if len(colors) > 1 else np.nan)
df.drop(['color'], axis=1, inplace=True)

In [402]:
df.head()

,animal_type,datetime,days_upon_outcome,is_mix,l_hair,m_hair,s_hair,is_adopted,sex_intact,sex,color_common,color_second
0,cat,735436,14,1,0,0,1,0,1,male,orange tabby,NaN
1,dog,735179,365,1,0,0,0,0,0,female,white,brown
2,dog,735387,365,0,0,0,0,1,0,male,blue,white
3,dog,735399,3285,1,0,0,0,0,0,male,white,NaN
5,dog,735148,120,1,0,0,0,0,1,male,brown,white


In [403]:
df['color_common'] = df['color_common'].str.replace(' ', '_')
df['color_second'] = df['color_second'].str.replace(' ', '_')

In [404]:
df.head()

,animal_type,datetime,days_upon_outcome,is_mix,l_hair,m_hair,s_hair,is_adopted,sex_intact,sex,color_common,color_second
0,cat,735436,14,1,0,0,1,0,1,male,orange_tabby,NaN
1,dog,735179,365,1,0,0,0,0,0,female,white,brown
2,dog,735387,365,0,0,0,0,1,0,male,blue,white
3,dog,735399,3285,1,0,0,0,0,0,male,white,NaN
5,dog,735148,120,1,0,0,0,0,1,male,brown,white


Я разделил цвета на первичный и второстепенный. Теперь я могу преобразовать наши категориальные данные при помощи OneHotEncoder. Все кроме "color_second", т.к. этот признак придётся проставлять отдельно ручками

Возьмём нашу функцию из смежной работы про Титаник

In [412]:
def one_hot_encode_new_columns(df: pd.DataFrame, col_name: str):
    enc = OneHotEncoder(categories='auto')
    
    encoded_data = enc.fit_transform(
        np.array( df[col_name] ).reshape(-1, 1)
    ).todense()
    
    encoded_feature_names = list(map(lambda val: re.sub(r'^.+_', f'{col_name}_', val), enc.get_feature_names()))
    
    return pd.DataFrame(data=encoded_data, columns=encoded_feature_names)

In [429]:
df = df.reset_index(drop=True)

ohe_col_names = ['animal_type', 'sex', 'color_common']

df_ohe = pd.concat([
    df,
    *map(lambda col_name: one_hot_encode_new_columns(df, col_name), ohe_col_names)
], sort=False, axis=1)

df_ohe.drop(ohe_col_names, axis=1, inplace=True)

df_ohe.head()

,datetime,days_upon_outcome,is_mix,l_hair,m_hair,s_hair,is_adopted,sex_intact,color_second,animal_type_bird,...,color_common_point,color_common_tabby,color_common_tan,color_common_torbie,color_common_tortie,color_common_point,color_common_tricolor,color_common_white,color_common_yellow,color_common_brindle
0,735436,14,1,0,0,1,0,1,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,735179,365,1,0,0,0,0,0,brown,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,735387,365,0,0,0,0,1,0,white,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,735399,3285,1,0,0,0,0,0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,735148,120,1,0,0,0,0,1,white,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [475]:
def set_second_color(row):
    second_color = row.get('color_second')

    if second_color:
        common_color_col_name = f'color_common_{second_color}'
        col_is_exist = row.get(common_color_col_name)
        if col_is_exist is not None:
            row[common_color_col_name] = 1

    return row

color_second_index = df_ohe[df_ohe.color_second.notna()].index

df_ohe.loc[color_second_index].apply(set_second_color, axis=1)

,datetime,days_upon_outcome,is_mix,l_hair,m_hair,s_hair,is_adopted,sex_intact,color_second,animal_type_bird,...,color_common_point,color_common_tabby,color_common_tan,color_common_torbie,color_common_tortie,color_common_point,color_common_tricolor,color_common_white,color_common_yellow,color_common_brindle
1,735179,365,1,0,0,0,0,0,brown,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,735387,365,0,0,0,0,1,0,white,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,735148,120,1,0,0,0,0,1,white,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,735459,30,1,0,0,1,1,1,white,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,735413,90,1,0,0,1,1,0,black,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56605,736726,730,1,0,0,1,1,0,tan,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56606,736726,30,0,0,0,0,1,0,white,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
56607,736726,30,0,0,0,0,1,0,white,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
56608,736726,1095,1,0,0,0,1,0,white,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [476]:
df_ohe.shape

(56611, 74)

In [477]:
df_ohe.drop(['color_second'], axis=1, inplace=True)

In [479]:
df_ohe.head()

,datetime,days_upon_outcome,is_mix,l_hair,m_hair,s_hair,is_adopted,sex_intact,animal_type_bird,animal_type_cat,...,color_common_point,color_common_tabby,color_common_tan,color_common_torbie,color_common_tortie,color_common_point,color_common_tricolor,color_common_white,color_common_yellow,color_common_brindle
0,735436,14,1,0,0,1,0,1,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,735179,365,1,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,735387,365,0,0,0,0,1,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,735399,3285,1,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,735148,120,1,0,0,0,0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Это было трудно, остаётся надеяться на хороший результат при тренировке модели

In [480]:
df_ohe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56611 entries, 0 to 56610
Data columns (total 73 columns):
datetime                  56611 non-null int64
days_upon_outcome         56611 non-null int64
is_mix                    56611 non-null int32
l_hair                    56611 non-null int32
m_hair                    56611 non-null int32
s_hair                    56611 non-null int32
is_adopted                56611 non-null int32
sex_intact                56611 non-null int32
animal_type_bird          56611 non-null float64
animal_type_cat           56611 non-null float64
animal_type_dog           56611 non-null float64
animal_type_livestock     56611 non-null float64
animal_type_other         56611 non-null float64
sex_female                56611 non-null float64
sex_male                  56611 non-null float64
sex_unknown               56611 non-null float64
color_common_agouti       56611 non-null float64
color_common_apricot      56611 non-null float64
color_common_black       

На опыте написания работы про Титаник, я уже научен тому, что Random Forest не работает с float значениями, поэтому приведу все значения к "int64" и "int32"

In [484]:
float_column_names = df_ohe.select_dtypes(float).columns

for col_name in float_column_names:
    df_ohe[col_name] = df_ohe[col_name].astype('int32')

In [485]:
df_ohe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56611 entries, 0 to 56610
Data columns (total 73 columns):
datetime                  56611 non-null int64
days_upon_outcome         56611 non-null int64
is_mix                    56611 non-null int32
l_hair                    56611 non-null int32
m_hair                    56611 non-null int32
s_hair                    56611 non-null int32
is_adopted                56611 non-null int32
sex_intact                56611 non-null int32
animal_type_bird          56611 non-null int32
animal_type_cat           56611 non-null int32
animal_type_dog           56611 non-null int32
animal_type_livestock     56611 non-null int32
animal_type_other         56611 non-null int32
sex_female                56611 non-null int32
sex_male                  56611 non-null int32
sex_unknown               56611 non-null int32
color_common_agouti       56611 non-null int32
color_common_apricot      56611 non-null int32
color_common_black        56611 non-null int3

Помимо появления возможности продолжать работать, теперь сам датасет в памяти занимает почти в два раза меньше места

Начнём тренировать модель

In [510]:
X = df_ohe.drop(['is_adopted'], axis=1)
Y = df_ohe['is_adopted']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [511]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((39627, 72), (16984, 72), (39627,), (16984,))

In [512]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [513]:
clf.score(X_test, Y_test)

0.7860927932171455

**Итог- точность модели составила примерно 78.1% - 78.6%**

Будет интересно посмотреть, что будет с нашей моделью, если я не буду добавлять второй признак цвета

In [497]:
df = df.reset_index(drop=True)

df_ohe_v2 = pd.concat([
    df,
    *map(lambda col_name: one_hot_encode_new_columns(df, col_name), ohe_col_names)
], sort=False, axis=1)

df_ohe_v2.drop(ohe_col_names, axis=1, inplace=True)

df_ohe_v2.head()

,datetime,days_upon_outcome,is_mix,l_hair,m_hair,s_hair,is_adopted,sex_intact,color_second,animal_type_bird,...,color_common_point,color_common_tabby,color_common_tan,color_common_torbie,color_common_tortie,color_common_point,color_common_tricolor,color_common_white,color_common_yellow,color_common_brindle
0,735436,14,1,0,0,1,0,1,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,735179,365,1,0,0,0,0,0,brown,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,735387,365,0,0,0,0,1,0,white,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,735399,3285,1,0,0,0,0,0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,735148,120,1,0,0,0,0,1,white,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [498]:
df_ohe_v2.drop('color_second', axis=1, inplace=True)

In [499]:
df_ohe_v2.head()

,datetime,days_upon_outcome,is_mix,l_hair,m_hair,s_hair,is_adopted,sex_intact,animal_type_bird,animal_type_cat,...,color_common_point,color_common_tabby,color_common_tan,color_common_torbie,color_common_tortie,color_common_point,color_common_tricolor,color_common_white,color_common_yellow,color_common_brindle
0,735436,14,1,0,0,1,0,1,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,735179,365,1,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,735387,365,0,0,0,0,1,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,735399,3285,1,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,735148,120,1,0,0,0,0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [517]:
X2 = df_ohe_v2.drop(['is_adopted'], axis=1)
Y2 = df_ohe_v2['is_adopted']

X2_train, X2_test, Y2_train, Y2_test = train_test_split(X2, Y2, test_size=0.3)

In [518]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X2_train, Y2_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [519]:
clf.score(X2_test, Y2_test)

0.7779674988224211

**Итог- точность модели составила примерно 77.7% - 78.5%**